# Stokes problem

In [1]:
import dolfinx
import dolfinx.fem
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import ufl

In [2]:
n = 10
m = 15
domain = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

In [3]:
tdim = domain.topology.dim
fdim = tdim - 1

num_cells = domain.topology.index_map(tdim).size_local
num_cells

300

In [4]:
boundary_entities = dolfinx.mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full((x.shape[1], ), True))
boundary_entities

array([  0,   2,   5,   7,  14,  16,  26,  28,  41,  43,  59,  61,  80,
        82, 104, 106, 131, 133, 161, 163, 166, 194, 196, 225, 227, 256,
       258, 287, 289, 318, 320, 347, 349, 373, 375, 396, 398, 416, 418,
       433, 435, 447, 449, 458, 460, 466, 468, 471, 473, 474], dtype=int32)

In [5]:
boundary_entities = boundary_entities[0:-1]

In [6]:
from ufl import (FiniteElement, Measure, VectorElement, MixedElement, SpatialCoordinate,
                 TestFunctions, TrialFunctions, div, grad, curl, sin, cos, exp, inner, FacetNormal)
from dolfinx import fem, io, mesh

In [7]:
k = 1
# V_el = FiniteElement("RT", domain.ufl_cell(), k)
V_el = VectorElement("CR", domain.ufl_cell(), k)
Q_el = FiniteElement("DG", domain.ufl_cell(), k-1)
W_el = MixedElement([V_el, Q_el])
W = fem.FunctionSpace(domain, W_el)
V, _ = W.sub(0).collapse()
Q, _ = W.sub(1).collapse()

In [8]:
uD = fem.Function(V)
pD = fem.Function(Q)

x = SpatialCoordinate(domain)

precursor = sin(ufl.pi*x[0])**2*sin(ufl.pi*x[1])**2
u_ex = curl(precursor)
p_ex = cos(ufl.pi*2*x[0])*cos(ufl.pi*2*x[1])

uD.interpolate(fem.Expression(u_ex, V.element.interpolation_points()))
pD.interpolate(fem.Expression(p_ex, Q.element.interpolation_points()))

f = -grad(p_ex)-div(grad(u_ex))

In [9]:
(u, p) = TrialFunctions(W)
(v, q) = TestFunctions(W)

dx = Measure("dx", domain)
ds = Measure("ds", domain)
# n = ufl.FacetNormal(domain)
# dx = ufl.dx
# ds = ufl.ds
a  = inner(grad(u), grad(v)) * dx + inner(p, div(v)) * dx 
a += inner(div(u), q) * dx
L = inner(f, v) * dx

In [10]:
# Now setup boundary conditions
boundary_dofs = dolfinx.fem.locate_dofs_topological((W.sub(0), V), fdim, boundary_entities)
bc = dolfinx.fem.dirichletbc(uD, boundary_dofs)

In [11]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={
                                   "ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})

In [ ]:
try:
    w_h = problem.solve()
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_h, p_h = w_h.split()

In [ ]:
u_h.name = 'u'
p_h.name = 'p'
uD.name = 'u_exact'
pD.name = 'p_exact'

In [ ]:
from dolfinx import io
# with io.VTXWriter(domain.comm, "output.bp", [w_h]) as vtx:
#     vtx.write(0.0)
with io.XDMFFile(domain.comm, "stokes.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(u_h)
    xdmf.write_function(uD)
    xdmf.write_function(p_h)
    xdmf.write_function(pD)


In [ ]:
import numpy as np

error_form_u = fem.form(inner(u_h-uD, u_h-uD) * dx)
error_local_u = fem.assemble_scalar(error_form_u)
errorL2_u = np.sqrt(domain.comm.allreduce(error_local_u, op=mpi4py.MPI.SUM))

error_form_p = fem.form(inner(p_h-pD, p_h-pD) * dx)
error_local_p = fem.assemble_scalar(error_form_p)
errorL2_p = np.sqrt(domain.comm.allreduce(error_local_p, op=mpi4py.MPI.SUM))

if domain.comm.rank == 0:
    print(fr"$L^2$-error u: {errorL2_u:.2e}")
    print(fr"$L^2$-error p: {errorL2_p:.2e}")